# Consolidación de tabla

In [1]:
import pandas as pd
import datetime
import numpy as np

## Leer tablas

In [20]:
dict_tablas = {datetime.datetime(2020, 3, 16): '../datos/tablas_originales/20200316_positivos.tsv',
               datetime.datetime(2020, 3, 17): '../datos/tablas_originales/20200317_positivos.tsv',
               datetime.datetime(2020, 3, 18): '../datos/tablas_originales/20200318_positivos.tsv',
               datetime.datetime(2020, 3, 19): '../datos/tablas_originales/20200319_positivos.tsv',
               datetime.datetime(2020, 3, 20): '../datos/tablas_originales/20200320_positivos.tsv',
               datetime.datetime(2020, 3, 21): '../datos/tablas_originales/20200321_positivos.tsv',
               datetime.datetime(2020, 3, 22): '../datos/tablas_originales/20200322_positivos.tsv',
               datetime.datetime(2020, 3, 23): '../datos/tablas_originales/20200323_positivos.tsv',
               datetime.datetime(2020, 3, 24): '../datos/tablas_originales/20200324_positivos.tsv',
               datetime.datetime(2020, 3, 25): '../datos/tablas_originales/20200325_positivos.tsv',
               datetime.datetime(2020, 3, 26): '../datos/tablas_originales/20200326_positivos.tsv',
               datetime.datetime(2020, 3, 27): '../datos/tablas_originales/20200327_positivos.tsv'}

In [21]:
# crea diccionario con objetos de tiemo: dataframe
def create_dict(dict_tablas):
    
    dict_df = {}
    
    for date in dict_tablas.keys():
        df = pd.read_csv(dict_tablas[date],
                         sep='\t',
                         parse_dates=['fecha_inicio_sintomas', 'fecha_llegada_mx'],
                         index_col=0)
        dict_df[date] = df
    
    return dict_df

In [22]:
dict_df = create_dict(dict_tablas)

## Consolidación simple

In [25]:
def merge_auto(dict_df):
    
    clean_dfs = []
    
    ordered_dates = sorted(dict_df.keys())

    for date in ordered_dates:
        
        # read table
        df = pd.read_csv(dict_tablas[date],
                         sep='\t',
                         parse_dates=['fecha_inicio_sintomas', 'fecha_llegada_mx'],
                         index_col=0)
        
        # add pseudo index
        df['pseudo_indice'] = date.strftime('%Y%m%d_') + df.index.astype(str)

        # si es la primera fecha
        if ordered_dates.index(date) == 0:
            # las nuevas, anadir fecha
            df['fecha_caso_nuevo'] = df['casos_nuevos'].replace(True, date)
            # todas las no nuevas, volverlas nan
            df['fecha_caso_nuevo'] = df['fecha_caso_nuevo'].replace(False, pd.NaT)
        else:
            # quitar las que no son nuevas
            df = df[df['casos_nuevos'] != False]

            # agregar fechas a las nuevas
            df['fecha_caso_nuevo'] = df['casos_nuevos'].replace(True, date)

        clean_dfs.append(df)
        df_all = pd.concat(clean_dfs)
        df_all = df_all.drop(columns=['casos_nuevos'])
        
        df_all = df_all[['pseudo_indice', 'estado', 'sexo', 'edad', 'fecha_inicio_sintomas', 
                         'id_rt-pcr', 'procedencia', 'fecha_llegada_mx', 'fecha_caso_nuevo']]
    
    return df_all

In [26]:
results = merge_auto(dict_df)

In [27]:
results

,pseudo_indice,estado,sexo,edad,fecha_inicio_sintomas,id_rt-pcr,procedencia,fecha_llegada_mx,fecha_caso_nuevo
1,20200316_1,Ciudad de México,M,35,2020-02-22,confirmado,Italia,2020-02-22,NaT
2,20200316_2,Sinaloa,M,41,2020-02-22,confirmado,Italia,2020-02-21,NaT
3,20200316_3,Ciudad de México,M,59,2020-02-23,confirmado,Italia,2020-02-22,NaT
4,20200316_4,Coahuila,F,20,2020-02-27,confirmado,Italia,2020-02-25,NaT
5,20200316_5,Chiapas,F,18,2020-02-25,confirmado,Italia,2020-02-25,NaT
...,...,...,...,...,...,...,...,...,...
713,20200327_713,Ciudad de México,M,37,2020-03-18,confirmado,Contacto,NaT,2020-03-27
714,20200327_714,Ciudad de México,M,65,2020-12-03,confirmado,España,NaT,2020-03-27
715,20200327_715,Ciudad de México,M,76,2020-03-15,confirmado,Contacto,NaT,2020-03-27
716,20200327_716,Ciudad de México,F,59,2020-09-03,confirmado,Contacto,NaT,2020-03-27


In [70]:
results.to_csv('../datos/tablas_procesadas/tabla_completa.tsv',
                sep='\t',
                index=None)

## Revisión de inconsistencias

In [78]:
def inconsistencias(dict_tablas):
    
    dict_df = {}
    
    for date in dict_tablas.keys():
        
        df = pd.read_csv(dict_tablas[date],
                         sep='\t',
                         parse_dates=['fecha_inicio_sintomas', 'fecha_llegada_mx'], index_col=0)
        
        dict_df[date] = df
    
    results = []
    
    ordered_dates = sorted(dict_df.keys())
    
    for date_1, date_2 in zip(ordered_dates, ordered_dates[1:]):

        df1 = dict_df[date_1]
        df2 = dict_df[date_2]

        # diferencia de longitud entre dos días
        len_diff = len(df2) - len(df1)

        # seleccionar solo casos nuevos en día 2
        nuevos = len(df2[df2['casos_nuevos'] == True])

        date_1_str = date_1.strftime('%d-%m-%Y')
        date_2_str = date_2.strftime('%d-%m-%Y')

        if len_diff == nuevos:
            print(f'{date_1_str} a {date_2_str}: ✓')

        else:
            print(f'{date_1_str} a {date_2_str}: inconsistente por {len_diff-nuevos}')


            # add dates
            df1['casos_nuevos'] = df1['casos_nuevos'].replace(True, date_1)
            df2['casos_nuevos'] = df2['casos_nuevos'].replace(True, date_2)

            # add pseudo index
            df1['pseudo_indice'] = date_1.strftime('%Y%m%d_') + df1.index.astype(str)
            df2['pseudo_indice'] = date_2.strftime('%Y%m%d_') + df2.index.astype(str)

            # check duplicates
            df_joint = pd.concat([df1, df2[df2['casos_nuevos'] == False]])


            df_joint['duplicado'] = df_joint.duplicated(subset=['estado', 'sexo', 'edad', 'fecha_inicio_sintomas', 'procedencia'], keep=False)

            df_inc = df_joint[df_joint['duplicado'] == False]
            df_inc.info = f'{date_1_str} a {date_2_str}: inconsistente por {len_diff-nuevos}'
            
            df_inc.to_csv('../datos/tablas_procesadas/err_log/{}-{}.tsv'.format(date_1_str, date_2_str),
                          sep='\t',
                          index=None)
            
            results.append(df_inc)
            
    return results

In [81]:
results = inconsistencias(dict_tablas)

16-03-2020 a 17-03-2020: ✓
17-03-2020 a 18-03-2020: ✓
18-03-2020 a 19-03-2020: ✓
19-03-2020 a 20-03-2020: inconsistente por -1
20-03-2020 a 21-03-2020: ✓
21-03-2020 a 22-03-2020: inconsistente por 4
22-03-2020 a 23-03-2020: ✓
23-03-2020 a 24-03-2020: ✓
24-03-2020 a 25-03-2020: ✓
25-03-2020 a 26-03-2020: inconsistente por -1
26-03-2020 a 27-03-2020: inconsistente por -1
